# This notebook handels data import and preprocessing

## Importing data

In [26]:
from typing import List

import numpy as np
import pandas as pd

#FILE PATHS
KAGGLE_DATASET_FOLDER = './kaggle_data'

#Restaurants
ACCEPTS = KAGGLE_DATASET_FOLDER + '/chefmozaccepts.csv'
CUISINE = KAGGLE_DATASET_FOLDER + '/chefmozcuisine.csv'
HOURS = KAGGLE_DATASET_FOLDER + '/chefmozhours4.csv'
PARKING = KAGGLE_DATASET_FOLDER + '/chefmozparking.csv'
PLACES = KAGGLE_DATASET_FOLDER + '/geoplaces2.csv'

#Consumers
USER_CUISINE = KAGGLE_DATASET_FOLDER + '/usercuisine.csv'
USER_PAYMENT = KAGGLE_DATASET_FOLDER + '/userpayment.csv'
USER_PROFILE = KAGGLE_DATASET_FOLDER + '/userprofile.csv'

#User-Item-Rating
RATINGS = KAGGLE_DATASET_FOLDER + '/rating_final.csv'

In [27]:
#IMPORTING DATASET

#Restaurants
res_accepts_df = pd.read_csv(ACCEPTS)
res_cuisine_df = pd.read_csv(CUISINE)
res_hours_df = pd.read_csv(HOURS)
res_parking_df = pd.read_csv(PARKING)
restaurants_df = pd.read_csv(PLACES)

#Consumers
user_cuisine_df = pd.read_csv(USER_CUISINE)
user_payment_df = pd.read_csv(USER_PAYMENT)
user_profile_df = pd.read_csv(USER_PROFILE)

#User-Item-Rating
ratings_df = pd.read_csv(RATINGS)

display(ratings_df)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


## Preprocessing Data

In [43]:
def selectData(dataframe: pd.DataFrame, columns: List[str] = None) -> pd.DataFrame:
    if columns is None:
        columns = [x for x in dataframe.columns]
    sub_df = dataframe[[x for x in columns]].copy()
    sub_df = sub_df.replace("?", np.NaN).dropna()
    sub_df = sub_df.reset_index()
    return sub_df


def selectUserData(columns: List[str] = None) -> pd.DataFrame:
    return selectData(user_profile_df, columns)

def selectRestData(columns: List[str] = None) -> pd.DataFrame:
    return selectData(restaurants_df, columns)


In [46]:
columns_profile = ['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
                    'dress_preference', 'marital_status', 'hijos', 'birth_year', 'activity', 'budget']
columns_restaurant = ['placeID', 'latitude', 'longitude',
                      'name', 'alcohol', 'smoking_area', 'dress_code', 'price']
dfu = selectUserData(columns_profile)
dfr = selectRestData(columns_restaurant)
display(dfr)

,index,placeID,latitude,longitude,name,alcohol,smoking_area,dress_code,price
0,0,134999,18.915421,-99.184871,Kiku Cuernavaca,No_Alcohol_Served,none,informal,medium
1,1,132825,22.147392,-100.983092,puesto de tacos,No_Alcohol_Served,none,informal,low
2,2,135106,22.149709,-100.976093,El Rinc�n de San Francisco,Wine-Beer,only at bar,informal,medium
3,3,132667,23.752697,-99.163359,little pizza Emilio Portes Gil,No_Alcohol_Served,none,informal,low
4,4,132613,23.752903,-99.165076,carnitas_mata,No_Alcohol_Served,permitted,informal,medium
...,...,...,...,...,...,...,...,...,...
125,125,132866,22.141220,-100.931311,Chaires,No_Alcohol_Served,not permitted,informal,medium
126,126,135072,22.149192,-101.002936,Sushi Itto,No_Alcohol_Served,none,informal,medium
127,127,135109,18.921785,-99.235350,Paniroles,Wine-Beer,not permitted,informal,medium
128,128,135019,18.875011,-99.159422,Restaurant Bar Coty y Pablo,No_Alcohol_Served,none,informal,low


Lenskit expects a ratings dataframe with user item and rating columns

In [20]:
# TODO: What to do with food_rating and service rating
ratings_df['sum_column'] = ratings_df['rating'] + \
    ratings_df['food_rating'] + ratings_df['service_rating']

ratings_df=ratings_df.rename(columns={'userID':'user','placeID':'item','rating':'general_rating','sum_column':'rating'})
display(ratings_df)


,user,item,general_rating,food_rating,service_rating,rating
0,U1077,135085,2,2,2,6
1,U1077,135038,2,2,1,5
2,U1077,132825,2,2,2,6
3,U1077,135060,1,2,2,5
4,U1068,135104,1,1,2,4
...,...,...,...,...,...,...
1156,U1043,132630,1,1,1,3
1157,U1011,132715,1,1,0,2
1158,U1068,132733,1,1,0,2
1159,U1068,132594,1,1,1,3


## Recommender System